In [8]:
import json
import accelerate
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


quantization_config = BitsAndBytesConfig(load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

In [16]:
article = ""
token_list = ''

with open("./distribution/real/part_0/0000.json", "r") as f:
    data = json.load(f)
    article = data["original_doc"]

with open("./json_files/cleaned_common_data.json", "r") as f:
    data = json.load(f)
    #only use tokens that are missing Wordpiece tokenization 
    token_list = [token for token in data if token["missing"] != "Wordpiece" and not token['item'].startswith('\\')]
    token_list = [token["item"] for token in token_list if len(token["item"]) > 1]
    token_list = ', '.join(token_list) # Join tokens with a comma



prompt = f'''
I will give you an article, enclosed in <article> tags and a list of tokens enclosed in <tokens> tags.
You will then generate a new article that is similar to the original article, but contains more occurrences of the tokens in the <tokens> tags. Make sure to use the tokens in a way that makes sense in the context of the article. The new article should be coherent and maintain the original meaning, but it should also include the tokens as much as possible.
The new article should not introduce new information, but rather rephrase the original article to include the tokens. The new article can and should be longer than the original article. Think of it as a creative writing task where you are asked to rewrite the article to include the tokens in a meaningful way.
Here is the original article and the tokens:
<article>{article}</article>
<tokens>{token_list}</tokens>
'''

model_inputs = tokenizer([prompt], return_tensors="pt").to("cpu")
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


'\nI will give you an article, enclosed in <article> tags and a list of tokens enclosed in <tokens> tags.\nYou will then generate a new article that is similar to the original article, but contains more occurrences of the tokens in the <tokens> tags. Make sure to use the tokens in a way that makes sense in the context of the article. The new article should be coherent and maintain the original meaning, but it should also include the tokens as much as possible.\nThe new article should not introduce new information, but rather rephrase the original article to include the tokens. The new article can and should be longer than the original article. Think of it as a creative writing task where you are asked to rewrite the article to include the tokens in a meaningful way.\nHere is the original article and the tokens:\n<article>Founder Journeys: The Personal and Professional Journey of SME Entrepreneurs \n\nFuzhou is a vibrant hub of entrepreneurial spirit and innovation, and stories of resil